In [1]:

from gensim.models import  doc2vec
import pandas as pd
import numpy as np

data = pd.read_csv("./DATA/training.csv", encoding='ISO8859-2')
test = pd.read_csv("./DATA/test.csv", encoding='ISO8859-2')
data.head(5).style

/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"Explanation Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
3,0001b41b1c6bb37e,"More I can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of types of accidents -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know. There appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0


In [2]:
data_all = pd.concat([data, test])

In [2]:
y_columns = data.columns[2:]
y_columns

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [3]:
def remove_strip(str):
    return str.replace("\r", "").replace("\n", "").strip()

def get_stop_words(f, encoding='utf-8'):
    stop_words = []
    with open(f, "r", encoding=encoding) as f_stopwords:
        for line in f_stopwords:
            line = remove_strip(line)
            stop_words.append(line.lower())
    stop_words = set(stop_words)
    print("total counts: ", len(stop_words))

    return stop_words

def read_corpus(fname, tokens_only=False, encoding='utf-8'):
    import gensim
    import smart_open
    with smart_open.open(fname, encoding=encoding) as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])
stop_words = get_stop_words("stopwords.txt")
# data_all['comment_text_clean'].to_csv("./DATA/lines.csv")
train_corpus = list(read_corpus("./DATA/lines.csv"))

total counts:  233


/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
model = doc2vec.Doc2Vec(vector_size=20000, window=4, min_count=35, epochs=10, seed=42)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [5]:
model.save(fname_or_handle="./DATA/doc2vec")

In [3]:
model = doc2vec.Doc2Vec.load(fname="./DATA/doc2vec")

In [4]:
train_doc2vec = model.dv.vectors[:127657]
test_doc2vec = model.dv.vectors[127657:]

train_df = pd.DataFrame(train_doc2vec)
train_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
0,0.000040,0.000047,0.000040,-0.000038,-0.000042,-0.000023,-0.000021,0.000023,-0.000033,-0.000042,...,-0.000007,0.000049,0.000045,-0.000023,0.000046,0.000003,-0.000012,0.000017,-0.000036,-0.000022
1,-0.000483,-0.000900,0.004402,-0.009081,-0.014446,0.009735,-0.000546,-0.005104,0.005535,-0.003893,...,-0.010594,0.007578,0.003607,0.005125,0.006250,0.001982,-0.003946,0.001351,-0.011819,-0.005357
2,0.001557,0.007516,-0.001297,-0.003258,-0.006896,-0.002670,-0.001690,-0.001826,0.004457,-0.002223,...,-0.000198,0.003150,-0.000395,0.002646,-0.002795,-0.003603,-0.002377,0.002033,-0.000099,0.002142
3,-0.010633,0.008425,0.001052,-0.007180,-0.007521,0.009944,-0.001933,0.004327,0.007524,-0.006032,...,-0.009257,-0.009474,-0.002200,0.008001,0.005278,0.010132,0.010924,0.003779,-0.013309,-0.002327
4,-0.015639,0.021219,0.012486,-0.019865,-0.003587,-0.005014,-0.014602,0.015337,0.009624,0.010447,...,0.013350,-0.007717,0.016971,0.005703,-0.007462,-0.009086,-0.005975,-0.007747,0.007631,-0.002217


In [5]:
del train_doc2vec
del model

In [6]:
from sklearn.model_selection import train_test_split
y = data['toxic']

In [15]:
train_df.loc[:, 0:20]

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.000040,0.000047,0.000040,-0.000038,-0.000042,-0.000023,-0.000021,0.000023,-0.000033,-0.000042,...,0.000049,-0.000023,0.000007,0.000044,0.000022,-0.000005,0.000047,-0.000030,0.000028,0.000020
1,-0.000483,-0.000900,0.004402,-0.009081,-0.014446,0.009735,-0.000546,-0.005104,0.005535,-0.003893,...,0.008390,-0.002746,-0.007831,-0.003512,0.003077,0.006789,-0.005752,0.004813,-0.001654,0.006974
2,0.001557,0.007516,-0.001297,-0.003258,-0.006896,-0.002670,-0.001690,-0.001826,0.004457,-0.002223,...,0.000078,0.000650,0.001475,-0.003079,-0.003559,-0.001850,-0.003219,-0.003403,0.006526,-0.000830
3,-0.010633,0.008425,0.001052,-0.007180,-0.007521,0.009944,-0.001933,0.004327,0.007524,-0.006032,...,-0.001479,-0.006915,0.007778,0.011296,0.001880,-0.006356,-0.005767,0.005237,0.000480,0.001613
4,-0.015639,0.021219,0.012486,-0.019865,-0.003587,-0.005014,-0.014602,0.015337,0.009624,0.010447,...,0.013868,0.010003,0.000679,0.007855,-0.009814,-0.003690,-0.017954,0.005277,-0.004219,-0.003766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127652,-0.000555,0.009785,-0.001923,-0.009518,-0.007267,0.005526,-0.012722,-0.002927,0.028967,0.016880,...,0.004161,-0.011312,0.000027,0.007001,0.010018,0.001193,-0.003227,-0.008497,-0.009833,-0.003798
127653,0.002812,0.024820,-0.033996,0.015426,-0.012184,0.000426,-0.001530,-0.004660,0.024888,0.005545,...,0.008329,0.000550,-0.013695,-0.001621,-0.005426,-0.004614,-0.010515,-0.015325,-0.028933,-0.013656
127654,-0.004708,0.009964,-0.003932,-0.000361,-0.002112,0.002400,-0.000918,0.001660,0.004699,0.008152,...,0.004081,-0.010665,0.001971,0.002411,-0.000859,-0.002428,0.000664,-0.004512,-0.001895,0.003898
127655,-0.004697,0.005901,-0.000093,0.003511,-0.003059,0.001944,0.003853,0.003096,0.002653,-0.000658,...,-0.001412,-0.002119,-0.000663,0.000381,-0.001680,-0.002318,-0.002244,-0.001480,0.001142,-0.001697


In [17]:
X_train, X_test, y_train, y_test = train_test_split(train_df.loc[:, :1000], y, test_size=0.25)

In [7]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(C=3.0, solver='sag', random_state=42)
classifier.fit(train_df[:60000], y[:60000])

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(C=3.0, random_state=42, solver='sag')

In [8]:
pred = classifier.predict(train_df[60000:])

from sklearn.metrics import f1_score
print('toxic-训练集F1值:%s'%f1_score(train_df[:60000], y[:60000]))
print('toxic-验证集F1值:%s'%f1_score(train_df[60000:], y[60000:]))

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

In [ ]:
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_over, y_over = smote.fit_resample(X_train, y_train)
classifier = LogisticRegression(C=3.0, solver='sag', random_state=42)
classifier.fit(X_over, y_over)
train_pred_toxic = classifier.predict(X_train)
test_pred_toxic = classifier.predict(X_test)
predict_toxic_0 = classifier.predict(test_doc2vec)
predict_toxic=pd.DataFrame(predict_toxic_0,columns=['toxic'])

from sklearn.metrics import f1_score
print('toxic-训练集F1值:%s'%f1_score(train_pred_toxic,y_train))
print('toxic-验证集F1值:%s'%f1_score(test_pred_toxic,y_test))









In [32]:
random_seed = 42
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score

models = []
i = 0
test_size = 0.25
rus = RandomUnderSampler(random_state=random_seed, )
X, y= rus.fit_resample(data_doc2vec, data[y_columns[i]])
X.shape

(24364, 300)

In [34]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_seed)
lgb =LGBMClassifier(max_depth=8, num_leaves=200, boosting_type='dart', random_state=random_seed)
lgb.fit(X_train, y_train)
y_train_pred = lgb.predict(X_train)
y_valid_pred = lgb.predict(X_valid)
print("train set f1 score: ", f1_score(y_train, y_train_pred))
print("training set %s f1_score: %.2f" % (y_columns[i], f1_score(y_valid, y_valid_pred) * 100))
models.append(lgb)

train set f1 score:  0.9697877118870648
training set toxic f1_score: 76.98
